## Machine Learning Exercise 2

In [1]:
import numpy as np

# Function to Generate an (m+1)-Dimensional Data Set

This function generates a dataset consisting of `m` continuous independent variables (X) and a binary dependent variable (Y). The relationship between X and Y is determined using a logistic function. To introduce noise to the labels (Y), we assume a Bernoulli distribution with a probability of flipping the label, determined by a parameter `θ`. The higher the value of `θ`, the greater the noise in the dataset.

## Parameters:

- `θ`: The probability of flipping the label (Y).
- `n`: The number of samples (size of the dataset).
- `m`: The number of independent variables.

## Outputs:

- `X`: An `n x (m+1)` numpy array of independent variable values, with the first column set to 1.
- `Y`: An `n x 1` numpy array of binary output values (0 or 1).
- `β`: A random vector of size `(m+1, 1)` representing the coefficients used to generate Y from X.

## Mathematical Definition:

The binary dependent variable `Y` is generated as follows:

$$
Y_i = 
\begin{cases} 
1 & \text{if} \ \frac{1}{1 + \exp(-\mathbf{x}_i \cdot \mathbf{\beta})} > 0.5 \\
0 & \text{otherwise}
\end{cases}
$$

Where:
- `X` is the matrix of independent variables.
- `β` is the coefficient vector.
- `θ` is the probability of flipping the label.

The labels are flipped with a probability `θ` to introduce noise into the dataset.

In [2]:
def generate_dataset(n, m, theta):
    #Generate X
    X = np.random.randn(n, m)
    X = np.hstack((np.ones((n, 1)), X))

    # Generate Coefficient Vector
    B = np.random.random(size=(m+1, 1))

    # Generate Y vector according to the given formula
    z = - np.dot(X, B)
    var = np.exp(z)
    intermediate_y = 1/(1 + var)
    Y = np.where(intermediate_y > 0.5, 1, 0)

    # Add Noise to Y
    # np.random.random makes a 1D n vector where values range 0 to 1 with equal prob
    # vector < theta represents 1 for all values when value is less than theta
    # effectively 1 occurs with theta probability in vector noise
    noise = np.random.random(n) < theta
    Y = np.abs(Y - np.reshape(noise, Y.shape))
    return X, Y, B

In [3]:
# Example usage:
theta = 0.1  # 10% probability of flipping labels
n = 100  # 100 samples
m = 5    # 5 independent variables

X, Y, B = generate_dataset(n, m, theta)

# Output the generated dataset and coefficients
X.shape, Y.shape, B.shape

((100, 6), (100, 1), (6, 1))

Write a function that learns the parameters of a logistic regression function given inputs:
- **X**: An n × m numpy array of independent variable values
- **Y**: The n × 1 binary numpy array of output values
- **k**: the number of iterations (epochs)
- **τ**: the threshold on change in Cost function value from the previous to current iteration
- **λ**: the learning rate for Gradient Descent

The function should implement the Gradient Descent algorithm as discussed in class that initializes **β** with random values and then updates these values in each iteration by moving in the direction defined by the partial derivative of the cost function with respect to each of the coefficients. The function should use only one loop that ends after a number of iterations (**k**) or a threshold on the change in cost function value (**τ**).

The output should be an **m + 1** dimensional vector of coefficients and the final cost function value.


In [4]:
def logistic_regression(X, Y, k, tau, lambda_):
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))
        
    def cost_function(X, Y, B):
        n = len(Y)
        intermediate = np.dot(X, B)
        predictions = sigmoid(intermediate)
        term1 = np.dot(Y.T, np.log(predictions)) # Dot product where first element of term is always transposed
        term2 = np.dot((1 - Y).T, np.log(1 - predictions))
        cost = -1/n * (term1 + term2)
        return cost
        
    n = X.shape[0]
    B = np.random.random(size=(X.shape[1], 1))
    Y = Y.reshape(-1, 1)  # Ensure Y is a column vector
    cost = cost_function(X, Y, B)
    
    for _ in range(k):
        prev_cost = cost
        
        # Gradient Descent
        predictions = sigmoid(np.dot(X, B))
        gradient = np.dot(X.T, (predictions - Y)) / n
        B = B - lambda_ * gradient

        cost = cost_function(X, Y, B)

        # Check threshold
        if abs(cost - prev_cost) < tau:
            break

    return B.flatten(), cost

Create a report investigating how differen values of n and θ impact the ability for your logistic regression
function to learn the coefficients, β, used to generate the output vector Y . Also include your derivation of
the partial derivative of the cost function with respect to the parameters of the model.

In [19]:
def evaluate_impact(n_values, theta_values, m=3):
    k = 100
    t = 0.01
    lambda_ = 0.1
    results = {}
    
    for n in n_values:
        for theta in theta_values:
            X, Y, true_B = generate_dataset(n, m, theta)
            estimated_B, final_cost = logistic_regression(X, Y, k, t, lambda_)
            
            # Calculate accuracy
            predictions = (np.dot(X, estimated_B) > 0.5).astype(int)
            accuracy = np.mean(Y.flatten() == predictions)
            
            results[(n, theta)] = {
                'accuracy': accuracy,
                'true_B': true_B.flatten(),
                'estimated_B': estimated_B
            }
    
    return results

n_values = [100, 1000, 10000]
theta_values = [0.1, 0.3, 0.5]

results = evaluate_impact(n_values, theta_values)

for (n, theta), result in results.items():
    print(f"n: {n}, θ: {theta}, Accuracy: {result['accuracy']:.2f}")
    print(f"True Beta: {result['true_B']}, Estimated Beta: {result['estimated_B']}\n")

n: 100, θ: 0.1, Accuracy: 0.77
True Beta: [0.08898067 0.60705857 0.88441801 0.6401544 ], Estimated Beta: [0.30384876 0.77322801 0.71606222 0.0207446 ]

n: 100, θ: 0.3, Accuracy: 0.55
True Beta: [0.37465786 0.3565255  0.96440219 0.14980055], Estimated Beta: [0.92370688 0.73078503 0.34191159 0.88323892]

n: 100, θ: 0.5, Accuracy: 0.41
True Beta: [0.24039413 0.40094347 0.61361472 0.12762783], Estimated Beta: [0.07727167 0.02827406 0.64145585 0.68088369]

n: 1000, θ: 0.1, Accuracy: 0.56
True Beta: [0.77435325 0.75022418 0.68664415 0.38872278], Estimated Beta: [0.14436469 0.47192996 0.63121849 0.51941654]

n: 1000, θ: 0.3, Accuracy: 0.53
True Beta: [0.08263231 0.87443636 0.09338647 0.15594453], Estimated Beta: [0.12740954 0.10668798 0.72624713 0.95884251]

n: 1000, θ: 0.5, Accuracy: 0.48
True Beta: [0.00446379 0.72002703 0.40057368 0.41914828], Estimated Beta: [0.22503169 0.88466273 0.58095721 0.57138275]

n: 10000, θ: 0.1, Accuracy: 0.69
True Beta: [0.80928504 0.23606458 0.77152841 0.81146

### Observations
As we can see that 

- accuracy is directly proportional to n
- accuracy is indirectly proportional to theta

### Derivation of the partial derivative of the cost function with respect to the parameters of the model.

The partial derivative of the logistic regression cost function (negative log-likelihood) with respect to the parameters ( \beta ) is given by:

$$nabla L(\beta) = X^T \cdot (h_\beta(X) - Y) $$

Where:

    ( h_\beta(X) = \frac{1}{1 + \exp(-X \cdot \beta)} ), the logistic function.
    ( X ) is the matrix of features.
    ( Y ) is the vector of observed labels.


Add L1 and L2 regularization to the Logistic Regression cost function. How does this impact the models
learnt? How does the choice of regularization constant impact the β vector learned?

In [10]:
import numpy as np

# Assuming `generate_dataset` is already defined

def logistic_regression(X, Y, k, tau, lambda_, regularization=None, alpha=0.1):
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))
        
    def cost_function(X, Y, B):
        n = len(Y)
        predictions = sigmoid(np.dot(X, B))
        term1 = np.dot(Y.T, np.log(predictions))
        term2 = np.dot((1 - Y).T, np.log(1 - predictions))
        base_cost = -1/n * (term1 + term2)
        
        if regularization == 'L1':
            reg_cost = alpha * np.sum(np.abs(B))
        elif regularization == 'L2':
            reg_cost = alpha / 2 * np.sum(B**2)
        else:
            reg_cost = 0

        return base_cost + reg_cost
    
    n = X.shape[0]
    B = np.random.random(size=(X.shape[1], 1))
    Y = Y.reshape(-1, 1)
    cost = cost_function(X, Y, B)
    
    for _ in range(k):
        prev_cost = cost

        predictions = sigmoid(np.dot(X, B))
        gradient = np.dot(X.T, (predictions - Y)) / n
        
        if regularization == 'L1':
            reg_gradient = alpha * np.sign(B)
        elif regularization == 'L2':
            reg_gradient = alpha * B
        else:
            reg_gradient = 0
        
        B = B - lambda_ * (gradient + reg_gradient)
        
        cost = cost_function(X, Y, B)

        if abs(cost - prev_cost) < tau:
            break

    return B.flatten(), cost

In [16]:
def evaluate_regularization(n=1000, m=3, theta=0.3, k=100, tau=0.01, lambda_=0.1):
    alpha_values=[0.01, 0.1, 1]
    regularization_methods = ['L1', 'L2', None]
    results = {}

    # Generate Data
    X, Y, true_B = generate_dataset(n, m, theta)

    for reg in regularization_methods:
        for alpha in alpha_values:
            estimated_B, final_cost = logistic_regression(X, Y, k, tau, lambda_, regularization=reg, alpha=alpha)
            predictions = (np.dot(X, estimated_B) > 0.5).astype(int)
            accuracy = np.mean(Y.flatten() == predictions)
            
            results[(reg, alpha)] = {
                'accuracy': accuracy,
                'estimated_Beta': estimated_B,
                'final_cost': final_cost.item()  # Ensure final_cost is a scalar
            }

    return results

results = evaluate_regularization()

for (reg, alpha), result in results.items():
    print(f"Regularization: {reg}, Alpha: {alpha}, Accuracy: {result['accuracy']:.2f}, Final Cost: {result['final_cost']:.2f}")
    print(f"Estimated Beta: {result['estimated_Beta']}\n")

Regularization: L1, Alpha: 0.01, Accuracy: 0.46, Final Cost: 0.68
Estimated Beta: [0.05465978 0.09095697 0.02414507 0.48102008]

Regularization: L1, Alpha: 0.1, Accuracy: 0.61, Final Cost: 0.88
Estimated Beta: [0.63430422 0.45244907 0.9619378  0.13119283]

Regularization: L1, Alpha: 1, Accuracy: 0.39, Final Cost: 0.88
Estimated Beta: [-0.04450389 -0.04049101 -0.0212851  -0.06338848]

Regularization: L2, Alpha: 0.01, Accuracy: 0.66, Final Cost: 0.65
Estimated Beta: [0.77195483 0.28874657 0.7265586  0.56620302]

Regularization: L2, Alpha: 0.1, Accuracy: 0.60, Final Cost: 0.69
Estimated Beta: [0.50474296 0.19653331 0.72598423 0.31086642]

Regularization: L2, Alpha: 1, Accuracy: 0.45, Final Cost: 0.71
Estimated Beta: [0.26125754 0.06358087 0.08856663 0.20331655]

Regularization: None, Alpha: 0.01, Accuracy: 0.63, Final Cost: 0.63
Estimated Beta: [0.72974585 0.60273043 0.18567731 0.37445455]

Regularization: None, Alpha: 0.1, Accuracy: 0.60, Final Cost: 0.65
Estimated Beta: [0.41163251 0.60

### Observations:
As We can see after comparing with No Regularization:

For L1 Regularization:
- The Model is showing lower accuracy in this case.

For L2 Regularization:
- The Model is showing higher accuracy than No Regularization But Only when `Alpha = 0.01`
- That is Alpha should be very low for making a logistic regression model of this dataset

In [17]:
class BaseRegression:
    """
    A base class for regression models.
    """
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None

    def fit(self, X, Y):
        raise NotImplementedError("Subclass must implement this method.")

    def predict(self, X):
        raise NotImplementedError("Subclass must implement this method.")


class LinearRegression(BaseRegression):
    """
    Linear Regression model class.
    """
    def fit(self, X, Y):
        n, m = X.shape
        self.weights = np.zeros(m)
        for _ in range(self.num_iterations):
            predictions = np.dot(X, self.weights)
            error = predictions - Y
            gradient = np.dot(X.T, error) / n
            self.weights -= self.learning_rate * gradient
    
    def predict(self, X):
        return np.dot(X, self.weights)


class LogisticRegression(BaseRegression):
    """
    Logistic Regression model class.
    """
    def __init__(self, learning_rate=0.01, num_iterations=1000, threshold=0.5):
        super().__init__(learning_rate, num_iterations)
        self.threshold = threshold

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, Y):
        n, m = X.shape
        self.weights = np.zeros(m)
        for _ in range(self.num_iterations):
            predictions = self.sigmoid(np.dot(X, self.weights))
            error = predictions - Y
            gradient = np.dot(X.T, error) / n
            self.weights -= self.learning_rate * gradient

    def predict(self, X):
        return (self.sigmoid(np.dot(X, self.weights)) >= self.threshold).astype(int)